# Sesión 3 - Cuaderno 2: Evidencia Práctica del "Data Skipping"

### **Introducción: De la Teoría a la Práctica de la Optimización**

En la Sesión 2, tomamos una decisión de diseño crucial: particionamos nuestras tablas de hechos (`pedidos_silver` y `devoluciones_silver`) por la columna `id_fecha`. La teoría nos dice que esto debería acelerar drásticamente cualquier consulta que filtre por un rango de fechas, ya que permite a Spark ignorar (o "saltarse") los datos que no son relevantes.

Hoy, vamos a dejar de confiar en la teoría y pasaremos a la **verificación empírica**. Usaremos el comando `EXPLAIN` que aprendimos en el cuaderno anterior para analizar el plan de ejecución y encontrar la prueba definitiva de que nuestra estrategia de particionamiento está funcionando como se esperaba.

**Objetivo:** Al finalizar este cuaderno, serás capaz de:

* Identificar en un plan de ejecución cuándo Spark está realizando un escaneo completo de tabla (`Full Scan`).
* Localizar y entender la sección `PartitionFilters` como la evidencia del "data skipping".
* Cuantificar el impacto del particionamiento en el rendimiento de una consulta.

### **Paso 1: Preparación del Entorno**

Como siempre, el primer paso es establecer el contexto de nuestra base de datos para asegurarnos de que estamos trabajando con las tablas correctas.

In [0]:
%sql
USE curso_arquitecturas;

### **Paso 2: El Escenario Base - Consulta Sin Filtro de Partición**

Para apreciar el beneficio del particionamiento, primero debemos establecer una línea base. Analizaremos el plan de una consulta que filtra la tabla `pedidos_silver` por una columna que **no es** la columna de partición (en este caso, `cantidad`).

Teóricamente, esperamos que Spark se vea forzado a leer todos los datos de la tabla porque no tiene forma de saber en qué particiones (carpetas) se encuentran los pedidos con una `cantidad` mayor a 4.

In [0]:
%sql
EXPLAIN SELECT *
FROM pedidos_silver
WHERE cantidad > 4;

In [0]:
%sql
SELECT COUNT(*)
FROM pedidos_silver
WHERE cantidad > 4;

In [0]:
%sql
SELECT *
FROM pedidos_silver
WHERE cantidad > 4
LIMIT 5000000;

### **Paso 3: Análisis del Plan - El Costoso "Full Scan"**

El resultado del plan físico se verá similar a esto:
```
== Physical Plan ==
*(1) ColumnarToRow
+- PhotonResultStage
+- PhotonScan parquet workspace.curso_arquitecturas.pedidos_silver [...] DataFilters: [isnotnull(cantidad#...), (cantidad#... > 4)], PartitionFilters: [], ...
```
Observa con atención el `PhotonScan`. Hay dos puntos clave en este plan:

1.  **`DataFilters: [..., (cantidad#... > 4)]`**: El optimizador ha aplicado correctamente el **Predicate Pushdown**. El filtro se aplica en el nivel más bajo posible.
2.  **`PartitionFilters: []`**: ¡Esta es la parte crucial! La lista de `PartitionFilters` está **vacía**. Esto nos confirma que Spark **no pudo eliminar ninguna partición** y, por lo tanto, se vio obligado a escanear todos los subdirectorios y todos los archivos de la tabla `pedidos_silver` para encontrar las filas que cumplían la condición. Esto es un **escaneo completo de tabla (Full Table Scan)**.

### **Paso 4: La Prueba Definitiva - Consulta Con Filtro de Partición**

Ahora, realicemos el experimento contrario. Ejecutaremos una consulta muy similar, pero esta vez filtraremos por la columna de partición `id_fecha`.

Según nuestra hipótesis, Spark debería ser lo suficientemente inteligente como para leer **únicamente** el subdirectorio correspondiente a la fecha '2023-01-15', ignorando todos los demás.

In [0]:
%sql
EXPLAIN SELECT *
FROM devoluciones_silver
WHERE id_fecha = '2023-01-15';

In [0]:
%sql
SELECT *
FROM devoluciones_silver
WHERE id_fecha = '2024-01-15';

In [0]:
%sql
SELECT *
FROM devoluciones_silver
WHERE id_fecha >= '2024-01-15' AND id_fecha <= '2024-06-15';

In [0]:
%sql
SELECT *
FROM pedidos_silver
WHERE id_fecha >= '2024-01-15' AND id_fecha <= '2024-06-15';

### **Paso 5: Análisis del Plan - "Data Skipping" en Acción**

El plan físico resultante será notablemente diferente:
```
== Physical Plan ==
*(1) ColumnarToRow
+- PhotonResultStage
+- PhotonScan parquet workspace.curso_arquitecturas.pedidos_silver [...] PartitionFilters: [isnotnull(id_fecha#...), (id_fecha#... = 2023-01-15)] ...
```
¡Bingo! El plan físico ha cambiado drásticamente.

* **`PartitionFilters: [..., (id_fecha#... = 2023-01-15)]`**: Esta es la prueba que estábamos buscando. La condición del `WHERE` ya no está en `DataFilters`, sino que ha sido promovida a `PartitionFilters`.

Esto le indica a Spark que no necesita abrir ni un solo archivo Parquet que no esté dentro del directorio `/id_fecha=2023-01-15/`. Si nuestra tabla tiene datos de cientos de días diferentes, esta consulta ha ignorado el 99% de los datos, resultando en una reducción masiva del I/O y un aumento exponencial en la velocidad.

### **Paso 6: Profundizando - Más Ejemplos de Filtros de Partición**

La eliminación de particiones no solo funciona con el operador de igualdad (`=`). El optimizador de Spark es lo suficientemente avanzado como para aplicarla en escenarios más complejos.

#### **Ejemplo 1: Filtrando con una lista de fechas (`IN`)**

¿Qué pasa si queremos analizar datos de dos días específicos?

**Análisis:** En el plan físico, verás que los `PartitionFilters` contendrán la condición `IN`. Spark es lo suficientemente inteligente como para escanear únicamente los dos directorios correspondientes a esas fechas, ignorando todo lo demás.

#### **Ejemplo 2: Filtrando con un rango de fechas (`BETWEEN`)**

Este es un caso de uso muy común en análisis de negocio, por ejemplo, para calcular las ventas de la última semana.

**Análisis:** El plan mostrará de nuevo que la condición `BETWEEN` se ha traducido a `PartitionFilters`. Spark escaneará únicamente las 7 particiones correspondientes a la primera semana de enero de 2024, haciendo la agregación mucho más rápida.

#### **Ejemplo 3: Filtrando con funciones de fecha**

¿Puede Spark seguir optimizando si usamos funciones en la columna de partición? ¡Sí!

**Análisis:** Aunque hemos aplicado funciones, Catalyst puede inferir que esta condición equivale a `id_fecha BETWEEN '2022-12-01' AND '2022-12-31'`. Por lo tanto, en los `PartitionFilters`, verás que Spark ha resuelto esto a un rango y escaneará únicamente las 31 particiones de diciembre de 2022.

In [0]:
%sql
SELECT * FROM devoluciones_silver 
WHERE id_fecha IN ('2025-02-01', '2024-03-15');

In [0]:
%sql
EXPLAIN SELECT * FROM devoluciones_silver 
WHERE id_fecha IN ('2023-02-01', '2023-03-15');

In [0]:
%sql
EXPLAIN SELECT motivo, COUNT(id_pedido) 
FROM devoluciones_silver 
WHERE id_fecha BETWEEN '2024-01-01' AND '2024-01-07'
GROUP BY motivo;

In [0]:
%sql
SELECT motivo, COUNT(id_pedido) 
FROM devoluciones_silver 
WHERE id_fecha BETWEEN '2024-01-01' AND '2024-01-07'
GROUP BY motivo;

In [0]:
%sql
EXPLAIN SELECT * FROM devoluciones_silver 
WHERE YEAR(id_fecha) = 2024 AND MONTH(id_fecha) = 12;

In [0]:
%sql
SELECT * FROM devoluciones_silver 
WHERE YEAR(id_fecha) = 2024 AND MONTH(id_fecha) = 12;

### **Conclusión**

Hemos verificado empíricamente que nuestra decisión de diseño de particionar la tabla `pedidos_silver` por `id_fecha` fue correcta. El comando `EXPLAIN` nos ha permitido visualizar el "data skipping" en acción a través de la sección `PartitionFilters` en múltiples escenarios.

Esta técnica es la optimización de rendimiento más importante para tablas de hechos grandes en un Data Warehouse, ya que la mayoría de las consultas analíticas se realizan sobre rangos de tiempo específicos. En el próximo cuaderno, exploraremos la otra técnica de optimización que aplicamos: **Z-Ordering**.